# General model for solving biot equation
This notebook contains a method for setting up and solving biot's equation:
\begin{align}
\nabla\cdot\epsilon - \alpha \nabla p = F \\
\frac{\partial}{\partial t} (\beta p + \alpha \nabla \cdot u ) - \nabla \cdot \nabla p = f
\end{align}
with boundary conditions on $\partial\Omega_d$ and $\partial\Omega_n$:
\begin{align}
p=p_b, \quad -K\nabla\cdot \boldsymbol{n} = v_b \\
\boldsymbol{u}= \boldsymbol{u}_b, \qquad\quad\boldsymbol{\epsilon}\cdot\boldsymbol{n} = t_b
\end{align}
where $\epsilon = \frac{1}{2}(\nabla u + (\nabla u)^T)

In the above, the primary variables are the displacements $\boldsymbol{u}$ and pressures $p$. The parameters are the scalar and vector source terms $F$ and $f$, the permeability tensor $K$, the stiffness matrix $D$, the Biot coefficient $\alpha$, the fluid compressibility $\beta$. $p_b$ is the pressure at the boundary with dirichlet condition, $v_b$ is the flux at the neumann boundary, $\boldsymbol{u}_b$ is the displacement at the dirichlet boundary, $t_b$ is the traction at the neumann boundary.

In [1]:
import numpy as np
import scipy.sparse as sps
import porepy as pp

from porepy.utils.derived_discretizations import implicit_euler as IE_discretizations

## Parent class to the biot equation
Since the pure mechanics problem essentially is one of the involved equations in the biot equations, we can use the pure mechanics class as parent to the Biot class.

In [ ]:
class Mechanics:
    """ Class for setting up a pure mechanics problem with no fractures.
    
    The following attributes are assigned to self:
    Method: __init__
        mesh_args (dict): Mesh sizes
        folder_name (str): Name of folder for visualization
        displacement_variable (str): Name of variable for displacement
        mechanics_parameter_keyword (str): Parameter keyword for mechanics problems
    Method: create_grid
        box (dict): Bounding box of domain.
        gb (pp.GridBucket): The grid bucket.
        Nd (int): Dimension of the domain.
    Method: rock_parameters
        lam (np.ndarray): lambda, Lamé parameter
        mu (np.ndarray): Lamé parameter
        
    The following attributes in self may be updated:
    Method: insert_heterogenity
        lam (np.ndarray)
        mu (np.ndarray)
        
    """

    def __init__(self, mesh_args, folder_name):
        """ Initialize the mechanics class.
        
        Parameters
        mesh_args (dict): Containing mesh sizes.
                For now, this should be e.g. {'n': 5}. 
                We use cartesian meshes.
        folder_name (str): Name of storage folder for visualization.
        
        """
        from porepy.geometry.geometry_property_checks import point_in_polygon
        self.mesh_args = mesh_args
        self.folder_name = folder_name
        
        # Variable
        self.displacement_variable = "u"
        
        # Keyword
        self.mechanics_parameter_keyword = "mechanics"
        
        
    def create_grid(self):
        """ Create a 2D domain without fractures.
        
        The method requires the following attribute:
            mesh_args (dict): Containing the mesh sizes
        
        The following attributes are assigned to self:
            box (dict): Bounding box of domain.
            gb (pp.GridBucket): The grid bucket.
            Nd (int): Dimension of the domain.
        
        """
        n = self.mesh_args['n']
        nx = np.array((n,n))
        self.box = {'xmin': 0, 'ymin': 0, 'xmax': 1, 'ymax': 1}
        
        #g = pp.CartGrid([n,n], physdims=[1,1])
        #g.compute_geometry()
        gb = pp.meshing.cart_grid([], nx, physdims=[1,1])
        gb.compute_geometry()
        gb.assign_node_ordering()
        
        self.gb = gb
        self.Nd = gb.dim_max()
        
    def domain_boundary_sides(self, g):
        """ 
        Obtain indices for faces on each side of the domain boundary.
        
        """
        tol = 1e-10
        box = self.box
        east = g.face_centers[0] > box["xmax"] - tol
        west = g.face_centers[0] < box["xmin"] + tol
        north = g.face_centers[1] > box["ymax"] - tol
        south = g.face_centers[1] < box["ymin"] + tol
        if self.Nd == 2:
            top = np.zeros(g.num_faces, dtype=bool)
            bottom = top.copy()
        else:
            top = g.face_centers[2] > box["zmax"] - tol
            bottom = g.face_centers[2] < box["zmin"] + tol
        
        all_bf = g.get_boundary_faces()
        return all_bf, east, west, north, south
        
    def bc_type(self, g):
        all_bf, *_ = self.domain_boundary_sides(g)
        bc = pp.BoundaryConditionVectorial(g, all_bf, "dir")
        return bc
    
    def bc_values(self, g):
        values = np.zeros((self.Nd, g.num_faces))
        values = values.ravel('F')
        return values

    def source(self, g):
        return np.zeros(g.dim * g.num_cells)
    
    def rock_parameters(self, g):
        """ Assemble the stress-strain tensor.
        
        The following attributes are assigned to self:
            lam (np.ndarray, (g.num_cells,)): lambda parameter
            mu (np.ndarray, (g.num_cells,)): mu parameter.

        """
        lam = np.ones(g.num_cells)
        mu = np.ones(g.num_cells)
        C = pp.FourthOrderTensor(mu, lam)
        self.lam = lam
        self.mu = mu
        return C
    
    def set_parameters(self):
        """ 
        Set default parameters to the matrix

        """
        for g, d in self.gb:
            if g.dim == self.Nd:
                
                # Rock parameters
                C = self.rock_parameters(g)
                
                # Boundary conditions and source value
                bc = self.bc_type(g)
                bc_val = self.bc_values(g)
                f = self.source(g)

                specified_parameters = {'fourth_order_tensor': C, 'source': f, 
                                        'bc': bc, 'bc_values': bc_val}
                pp.initialize_default_data(g, d, self.mechanics_parameter_keyword, 
                                           specified_parameters)
        
    def insert_heterogenity(self, physdims, center, mu, lam, tol=1e-8):
        """ Insert a heterogenity to the domain.

        A heterogenity in this sense means a rectangle with
        a different mu and/or lambda (e.g. different C) than
        the surrounding matrix.
        Assumes that parameters are already been set. Assumes existense
        of attributes self.lam and self.mu

        Parameters:
        physdims (np.ndarray, (2,) ): Physical dimensions of rectangle
        center (np.ndarray, (2,)): Coordinates of center of rectangle
        mu (float): Value of mu in the rectangle.
        lam (float): Value of lambda in the rectangle.
        tol (float, Default: 1e-8): Tolerance of rectangle bounds.

        The following attributes in self are updated:
            mu (np.ndarray, (g.num_cells,)): mu parameter
            lam (np.ndarray, (g.num_cells,)): Lambda parameter

        In addition, the parameter dictionary for the domain of highest dimension
        (we assume no fracs) has its entry 'fourth_order_tensor' updated

        """
        from porepy.geometry.geometry_property_checks import point_in_polygon
        g = self.gb.grids_of_dimension(self.Nd)[0]
        data = self.gb.node_props(g)
        R = RectangleByCenter(physdims, center)

        # This code assumes 2D.
        nds = np.copy(g.cell_centers)
        nds[2] = np.array(range(g.num_cells)) # Use z-coordinate as index.
        rect_nodes = nds[2, point_in_polygon(R.nodes, nds[[0,1],:], tol=tol)].astype(int)

        self.mu[rect_nodes] = mu
        self.lam[rect_nodes] = lam
        C = pp.FourthOrderTensor(self.mu, self.lam)

        data[pp.PARAMETERS]['fourth_order_tensor'] = C
        
    def assign_variables(self):
        """ 
        Assign variables to the node of the grid bucket.

        """
        gb = self.gb
        for g, d in gb:
            if g.dim == self.Nd:
                d[pp.PRIMARY_VARIABLES] = {
                    self.displacement_variable: {"cells": self.Nd}
                }
            else:
                d[pp.PRIMARY_VARIABLES] = {}
        
        for e, d in gb.edges():
            
            if e[0].dim == self.Nd:
                pass
            
            d[pp.PRIMARY_VARIABLES] = {}
    
    def assign_discretization(self):
        """ 
        Assign discretization to the node of the grid bucket
        
        """
        # We solve the highest dimension problem using mpsa.
        # There should be no other dimensions present.
        Nd = self.Nd
        gb = self.gb
        mpsa = pp.Mpsa(self.mechanics_parameter_keyword)
        
        for g, d in gb:
            if g.dim == Nd:
                d[pp.DISCRETIZATION] = {
                    self.displacement_variable: {'mpsa': mpsa}}
        
        # Assume no edges, so we don't loop over them.
        

In [ ]:
class Biot(Mechanics):
    
    def __init__(self, mesh_args, folder_name, **kwargs):
        super().__init__(mesh_args, folder_name)
        
        # Variables
        self.displacement_variable = 'u'
        self.scalar_variable = 'p'
        
        self.mechanics_parameter_key = 'mechanics'
        self.scalar_parameter_key = 'flow'
        
        # Scaling coefficients
        self.scalar_scale = 1
        self.length_scale = 1
        
        # Time stepping
        self.time_step = kwargs.get('time_step', 1)
        
    def bc_type_mechanics(self, g)
        """ Use parent class for bc"""
        return super().bc_type(g)
    
    def bc_type_scalar(self, g):
        all_b, *_ = self.domain_boundary_sides(g)
        return pp.BoundaryCondition(g, all_bf, 'dir')
    
    def bc_values_mechanics(self, g):
        return super().bc_values(g)
    
    def source_mechanics(self, g):
        return super().source(g)

    def source_scalar(self, g):
        return np.zeros(g.num_cells)
    
    def biot_alpha(self):
        return 1
    
    def set_parameters(self):
        """
        Set the parameters for the model
        """
        self.set_scalar_parameters()
        self.set_mechanics_parameters()
    
    def rock_parameters(self, g):
        """ Assemble the stress-strain tensor.
        
        Overwrites parent to account for scales.
        
        """
        lam = np.ones(g.num_cells) / self.scalar_scale
        mu = np.ones(g.num_cells) / self.scalar_scale
        C = pp.FourthOrderTensor(mu, lam)
        self.lam = lam
        self.mu = mu
        return C
        
    def set_mechanics_parameters(self):
        """ Set the mechanics related parameters for the model"""
        gb = self.gb
        
        for g, d in gb:
            if g.dim == self.Nd:
                
                # Rock parameters
                C = self.rock_parameters(g)
                
                # Boundary conditions and source term
                bc = self.bc_type_mechanics(g)
                bc_val = self.bc_values_mechanics(g)
                source = self.source_mechanics(g)
                
                specified_parameters = {
                    'fourth_order_tensor': C,
                    'bc': bc,
                    'bc_values': bc_val,
                    'source': source,
                    'time_step': self.time_step,
                    'biot_alpha': self.biot_alpha(),
                }
                
                pp.initialize_default_data(g, d,
                                           self.mechanics_parameter_key,
                                           specified_parameters,
                                          )
                
    def set_scalar_parameters(self):
        gb = self.gb
        
        tensor_scale = self.scalar_scale / self.length_scale ** 2
        kappa = 1 * tensor_scale
        mass_weight = 1